In [34]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict,Annotated
from pydantic import BaseModel, Field
load_dotenv()
model=ChatOpenAI(model="gpt-4o-mini")
import operator # to use the reducer function


In [35]:
# define the schema for strucured output
class EvaluationSchema(BaseModel): 
    feedback:str=Field(description="Detailed feedback on the essay provided.")
    score:int=Field(description="score out of 10",ge=0,le=10)


In [36]:
#defining the structured model
Structured_model=model.with_structured_output(EvaluationSchema)

In [37]:
essay="""" Black holes are among the most fascinating and enigmatic phenomena in the universe. Predicted by Einstein’s theory of general relativity and confirmed through modern observations, black holes challenge our understanding of space, time, and the fundamental laws of physics. Though invisible by nature, their powerful influence on surrounding matter makes them some of the most studied objects in modern astrophysics.

What Is a Black Hole?

A black hole is a region of space where gravity is so strong that nothing—not even light—can escape from it. This immense gravitational pull arises when a large amount of mass is compressed into an extremely small area. At the center of a black hole lies the singularity, a point where density becomes infinite and the known laws of physics break down. Surrounding the singularity is the event horizon, the boundary beyond which escape is impossible.

How Do Black Holes Form?

Most black holes form from the remnants of massive stars. When such a star exhausts its nuclear fuel, it can no longer support itself against gravity and collapses inward, triggering a supernova explosion. If the remaining core is sufficiently massive, it collapses completely, forming a black hole. In addition to these stellar-mass black holes, scientists have discovered supermassive black holes—millions or even billions of times more massive than the Sun—at the centers of most galaxies."""

In [38]:
'''prompt=f'Evaluate the language quality of the following essay and give it a score out of 10:\n\n{essay}'
result=Structured_model.invoke(prompt)
print(result.score)'''

"prompt=f'Evaluate the language quality of the following essay and give it a score out of 10:\n\n{essay}'\nresult=Structured_model.invoke(prompt)\nprint(result.score)"

In [39]:
#define the state now
class UPSCState(TypedDict):
    essay:str
    language_feedback:str
    analysis_feedback:str
    clarity_feedback:str
    overall_feedback:str
    individual_scores:Annotated[list[int],operator.add]
    average_score:float
    

In [40]:
def evaluate_language(state:UPSCState)->UPSCState:
    prompt=f'Evaluate the language quality of the following essay and give it a score out of 10:\n\n{state["essay"]}'
    output=Structured_model.invoke(prompt)

    return {'language_feedback':output.feedback,'individual_scores':[output.score]}

In [41]:
def evaluate_analysis(state:UPSCState)->UPSCState:
    prompt=f'Evaluate the depth of analysis of the following essay and give it a score out of 10:\n\n{state["essay"]}'
    output=Structured_model.invoke(prompt)

    return {'analysis_feedback':output.feedback,'individual_scores':[output.score]}

In [42]:
def evaluate_thought(state:UPSCState)->UPSCState:
    prompt=f'Evaluate the clarity of thought in the following essay and give it a score out of 10:\n\n{state["essay"]}'
    output=Structured_model.invoke(prompt)

    return {'clarity_feedback':output.feedback,'individual_scores':[output.score]}

In [43]:
def final_evaluation(state:UPSCState)->UPSCState:
    #summary feedback
    prompt=f'Based on the following feedbacks, provide an overall feedback for the essay:\n\nLanguage Feedback: {state["language_feedback"]}\n\nAnalysis Feedback: {state["analysis_feedback"]}\n\nClarity Feedback: {state["clarity_feedback"]}'
    output=model.invoke(prompt)

    #avg calculation score
    avg_score=sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback':output.content,'average_score':avg_score}

In [44]:
graph=StateGraph(UPSCState)

graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)

graph.add_edge(START,'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_thought')
graph.add_edge('evaluate_language','final_evaluation')
graph.add_edge('evaluate_analysis','final_evaluation')
graph.add_edge('evaluate_thought','final_evaluation')
graph.add_edge('final_evaluation',END)

workflow=graph.compile()

In [ ]:
initial_state={
    'essay':essay
}   
#workflow.invoke(initial_state) gives the all the feedback in the dictionary
final_state=workflow.invoke(initial_state)

print("Overall Feedback:",final_state['overall_feedback'])
print("Average Score:",final_state['average_score'])



Overall Feedback: Overall Feedback: 

The essay provides a solid foundational overview of black holes, effectively defining the concept and explaining their formation while maintaining a logical structure. The language used is clear and accessible, allowing readers to grasp complex ideas without overwhelming them with technical details. However, there is a notable opportunity for enhancement in both engagement and depth of analysis.

To increase engagement, consider incorporating more compelling hooks in the introduction and a stronger conclusion that reinforces the essay's key points. Including real-world implications or intriguing anecdotes could help captivate the audience further. Additionally, varying sentence structures throughout the text could improve flow and keep readers more engaged.

In terms of depth, while the essay successfully introduces fundamental concepts such as singularity and event horizons, it lacks a thorough exploration of the implications of black holes in our